In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import lxml
from html.parser import HTMLParser
import re

In [18]:
url = "https://fi.wikipedia.org/wiki/Helsinki"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table') # in html table is represented by the tag <table>

# we can see how many tables were found by checking the length of the tables list
len(tables)

44

In [20]:
for index,table in enumerate(tables):
    if ("Väkiluku" in str(table)): # There may be many tables with the same word, print out to check
        table_index = index
        print(table_index)

0
19
22


In [23]:
# Check what each table[table_index] contains
str(tables[22])

'<table class="wikitable sortable">\n<tbody><tr>\n<th>#</th>\n<th>Taajama</th>\n<th>Väkiluku<br/>(31.12.2018)\n</th></tr>\n<tr>\n<td>1</td>\n<td><b><a href="/wiki/Helsingin_keskustaajama" title="Helsingin keskustaajama">Helsingin keskustaajama</a></b>*</td>\n<td align="right">630 603\n</td></tr>\n<tr>\n<td>2</td>\n<td><a href="/wiki/S%C3%B6derkulla" title="Söderkulla">Söderkulla</a>*</td>\n<td align="right">1 758\n</td></tr>\n<tr>\n<td>3</td>\n<td><a href="/wiki/Suomenlinna_(kaupunginosa)" title="Suomenlinna (kaupunginosa)">Suomenlinna</a></td>\n<td align="right">707\n</td></tr></tbody></table>'

In [24]:
print(tables[19].prettify())

<table class="wikitable">
 <tbody>
  <tr>
   <th colspan="3">
    Väkiluku
   </th>
  </tr>
  <tr>
   <th>
    Vuosi
   </th>
   <th>
    Helsinki
   </th>
   <th>
    Pääkaupunkiseutu
   </th>
  </tr>
  <tr>
   <td align="right">
    1875
   </td>
   <td align="right">
    <span style="display:none">
     &amp;&amp;&amp;&amp;&amp;&amp;&amp;&amp;&amp;&amp;023000.&amp;&amp;&amp;&amp;00
    </span>
    23 000
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td align="right">
    1900
   </td>
   <td align="right">
    <span style="display:none">
     &amp;&amp;&amp;&amp;&amp;&amp;&amp;&amp;&amp;&amp;079000.&amp;&amp;&amp;&amp;00
    </span>
    79 000
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td align="right">
    1910
   </td>
   <td align="right">
    <span style="display:none">
     &amp;&amp;&amp;&amp;&amp;&amp;&amp;&amp;&amp;0119000.&amp;&amp;&amp;&amp;00
    </span>
    119 000
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td align="right">
    1920
   </td>
   <td align="right">
   

In [55]:
population_data = pd.DataFrame(columns=["Vuosi", "Helsinki", "Pääkaupunkiseutu"])

numbers = re.compile(r'\d+(?:\.\d+)?') # poista muu paitsi numerot, esim ampersandit ja pisteet

for row in tables[19].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        # perusmuoto: year = col[0].text ja siihen lisätään strip sun muuta jos tarpeen
        # numbers on regex compiler joka poistaa ikävimpiä hässäköitä esim jos numerot on kahteen kertaan
        year = col[0].text.strip() # strip poistaa beginning and trailing esim \n
        year = year.lstrip("0")
        year = year.lstrip("0")
        city = numbers.findall(col[1].text)
        city = city[0]
        area = numbers.findall(col[2].text)
        if area !=[]:
            area = area[0].lstrip("0")
        else:
            area = ""
        population_data = population_data.append({"Vuosi":year, "Helsinki":city, "Pääkaupunkiseutu":area}, ignore_index=True)

population_data

,Vuosi,Helsinki,Pääkaupunkiseutu
0,1875,023000,
1,1900,079000,
2,1910,0119000,
3,1920,0152000,
4,1930,0206000,
5,1940,0252000,307000
6,1950,0369000,408000
7,1960,0448000,543000
8,1970,0524000,693000
9,1980,0484000,755000


In [56]:
pd.read_html(str(tables[19]), flavor='bs4')

[   Väkiluku                          
       Vuosi Helsinki Pääkaupunkiseutu
 0      1875   23 000              NaN
 1      1900   79 000              NaN
 2      1910  119 000              NaN
 3      1920  152 000              NaN
 4      1930  206 000              NaN
 5      1940  252 000          307 000
 6      1950  369 000          408 000
 7      1960  448 000          543 000
 8      1970  524 000          693 000
 9      1980  484 000          755 000
 10     1990  491 000          822 000
 11     2000  551 000          949 000
 12     2010  588 343        1 038 000]

In [66]:
# etsi kaikkien päiväkotien osoitteet Helsingin kaupungin verkkosivuilta

url = "https://www.hel.fi/varhaiskasvatus/fi/paivakodit/aada/aada"

data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

for el in soup.find_all(id="contact-details"):
    addresses = el.find("p")
    print(addresses)
#    "id="contact-details""

<p>
								  Ida Aalbergin tie 8<br/>
								  00400 Helsinki<br/>
								   
								  
								   +358 9 310 41523<br/>
								   
								  <!-- a href="/helsinki/fi/kaupunki-ja-hallinto/osallistu-ja-vaikuta/ota-yhteytta/hae-yhteystietoja/toimipistekuvaus?id=60870">LisÃ¤Ã¤ toimipisteestÃ¤</a -->
								  <!--
										<a href="/helsinki/fi/kaupunki-ja-hallinto/osallistu-ja-vaikuta/ota-yhteytta/hae-yhteystietoja/toimipistekuvaus?id=60870">Lue lis&#228&#228</a><br/>
								  -->
								  
										<a href="https://palvelukartta.hel.fi/unit/60870" target="_self"><span class="service-link-icon sprites small_icon_palvelut_kartalla"></span> Toimipiste kartalla</a><br/>
								  
								</p>
